In [1]:
import numpy as np
import pandas as pd
import json
import re

import requests
from bs4 import BeautifulSoup

import plotly.express as px

In [2]:
pd.options.display.max_columns=None
pd.options.display.max_rows=None
pd.options.display.max_colwidth=250
pd.options.display.max_seq_items=500

In [4]:
# Import card data
# Data sourced from: https://mtgjson.com/api/v5/AllPrintingsCSVFiles.zip
card_dtypes = {
    'colors':np.object,
    'faceConvertedManaCost':np.object,
    'flavorText': np.object,
    'frameEffects': np.object,
    'leadershipSkills': np.object,
    'name': np.object,
    'text': np.object,
}

df_base = pd.read_csv("./resources/cards.csv",dtype=card_dtypes,low_memory=False)

# Remove key rows
df_base = df_base[(df_base.isOnlineOnly == 0)]
df_base = df_base[(df_base.isOversized == 0)]
df_base = df_base[(df_base.isPromo == 0)]
df_base = df_base[~(df_base.layout == 'vanguard')]


# Keep fields likely to support data feature build
df = df_base[[
    'index',
    'id',
    'colorIdentity',
    'colorIndicator',
    'colors',
    'convertedManaCost',
    'faceConvertedManaCost',
    'faceName',
    'flavorText',
    'hand',
    'hasAlternativeDeckLimit',
    'isOnlineOnly',
    'isOversized',
    'isPromo',
    'isReprint',
    'isReserved',
    'isStarter',
    'isTextless',
    'keywords',
    'layout',
    'leadershipSkills',
    'life',
    'loyalty',
    'manaCost',
    'multiverseId',
    'name',
    'number',
    'otherFaceIds',
    'power',
    'printings',
    'rarity',
    'setCode',
    'side',
    'subtypes',
    'supertypes',
    'text',
    'toughness',
    'type',
    'types',
    'uuid',
    'variations',
    'watermark'
]].copy()

############################################################
############################################################

# Create unique row per card name / allowing for multiple faces (i.e. names may be duplicated)
# 'side' needs to be filled in or groupby portion of statement doesn't work properly
df['side'].fillna('normal',inplace=True)
df['name_row'] = df.sort_values(by='id',ascending=True).groupby(['name','side']).cumcount() + 1
df = df[(df['name_row'] == 1)]

# Flag double layout cards
df['double_layout'] = 1
df['double_layout'].where(df['layout'].isin(['transform','split','adventure','modal_dfc','flip','aftermath','meld']),0,inplace=True)


############################################################
############################################################


# Add in mana cost counts
df['manaCost_NA'] = df.manaCost.isna()*1 # Column to flag NA values for manaCost
df['manaCost'].fillna('{none}',inplace=True) # Use '{none}' in lower case, since all other manaCost letters in upper case.
df['manaCost_Generic_count'] = np.where(df.manaCost.str.contains('\{[\d]+?\}'),df.manaCost.str.extract('\{([\d]+?)\}',expand=False),0)
df['manaCost_W_count'] = df.manaCost.str.count('{W}')
df['manaCost_U_count'] = df.manaCost.str.count('{U}')
df['manaCost_B_count'] = df.manaCost.str.count('{B}')
df['manaCost_R_count'] = df.manaCost.str.count('{R}')
df['manaCost_G_count'] = df.manaCost.str.count('{G}')
df['manaCost_C_count'] = df.manaCost.str.count('{C}')
df['manaCost_WP_count'] = df.manaCost.str.count('{W/P}')
df['manaCost_UP_count'] = df.manaCost.str.count('{B/P}')
df['manaCost_BP_count'] = df.manaCost.str.count('{U/P}')
df['manaCost_RP_count'] = df.manaCost.str.count('{R/P}')
df['manaCost_GP_count'] = df.manaCost.str.count('{G/P}')
df['manaCost_H_WU_count'] = df.manaCost.str.count('{W/U}')
df['manaCost_H_UB_count'] = df.manaCost.str.count('{U/B}')
df['manaCost_H_BR_count'] = df.manaCost.str.count('{B/R}')
df['manaCost_H_RG_count'] = df.manaCost.str.count('{R/G}')
df['manaCost_H_GW_count'] = df.manaCost.str.count('{G/W}')
df['manaCost_H_WB_count'] = df.manaCost.str.count('{W/B}')
df['manaCost_H_UR_count'] = df.manaCost.str.count('{U/R}')
df['manaCost_H_BG_count'] = df.manaCost.str.count('{B/G}')
df['manaCost_H_RW_count'] = df.manaCost.str.count('{R/W}')
df['manaCost_H_GU_count'] = df.manaCost.str.count('{G/U}')
df['manaCost_H_2W_count'] = df.manaCost.str.count('{2/W}')
df['manaCost_H_2U_count'] = df.manaCost.str.count('{2/U}')
df['manaCost_H_2B_count'] = df.manaCost.str.count('{2/B}')
df['manaCost_H_2R_count'] = df.manaCost.str.count('{2/R}')
df['manaCost_H_2G_count'] = df.manaCost.str.count('{2/G}')
df['manaCost_X_count'] = df.manaCost.str.count('{X}')
df['manaCost_Y_count'] = df.manaCost.str.count('{Y}')
df['manaCost_Z_count'] = df.manaCost.str.count('{Z}')
df['manaCost_Snow_count'] = df.manaCost.str.count('{S}')
df['manaCost_HW_count'] = df.manaCost.str.count('{HW}')

############################################################
############################################################

# OneHot Encode all keywords in the data, and add some other info related to keywords
df['keywords_NA'] = df.keywords.isna()*1
df.keywords.fillna('{none}',inplace=True)
df['keywords_count'] = [len(i) for i in df.keywords.str.split(',').tolist()] * np.where(df.keywords_NA,0,1)


all_keywords = df.keywords.str.split(",").tolist()
unique_keywords = []

for i in all_keywords:
    for j in i:
        if j != '{none}':
            j.capitalize()
            unique_keywords.append(j)
unique_keywords = set(unique_keywords)
unique_keywords = list(unique_keywords)
unique_keywords.sort()

for keyword in unique_keywords:
    col_name = 'keyword_' + keyword.replace(' ','_')
    df[col_name] = df.keywords.str.contains(keyword) * 1

############################################################
############################################################


# Other effects ############################################################

df['text_NA'] = df.keywords.isna()*1
df.text.fillna('{none}',inplace=True)

# Drawing cards (generally a benefit)
df_draw_cards = df.text.str.extract('[Dd]raw(?!\s[Ss]tep)\s(.*?)card?')
df_draw_cards.rename({0:'extract_text'},axis=1,inplace=True)
df_draw_cards.fillna(0,inplace=True)

cond = [
    df_draw_cards['extract_text'].str[0] == 'a',
    df_draw_cards['extract_text'].str[0:3] == 'two',
    df_draw_cards['extract_text'].str[0:5] == 'three',
    df_draw_cards['extract_text'].str[0:4] == 'four',
    df_draw_cards['extract_text'].str[0:4] == 'five',
    df_draw_cards['extract_text'].str[0:3] == 'six',
    df_draw_cards['extract_text'].str[0:5] == 'seven',
    df_draw_cards['extract_text'].str[0:5] == 'eight',
    df_draw_cards['extract_text'].str[0:4] == 'nine',
    df_draw_cards['extract_text'].str[0:5] == 'half X',
    df_draw_cards['extract_text'].str[0:0] == 'X'
]

output = [1,2,3,4,5,6,7,8,9,15,20]

df['effect_draw_cards'] = np.select(cond,output,default=0)

# Beneficial discards (i.e. apply to opponent / target player - since generally you choose your opponent, unless you're drawing cards)
df_extract = (df.text.str.contains('[Tt]arget\s(opponent|player).*[Dd]iscards\s.*?card?',case=False))
df['effect_discard_target_player'] = df_extract*1

# Discard own cards (cost)
df_extract = (df.text.str.contains('Discard\s.*?card?',case=False))
df['effect_discard_own_cards'] = df_extract*1

# Loot ability dummy variable
df_extract = (df.text.str.contains('[Dd]raw a card, then discard a card',case=False))
df['effect_loot'] = df_extract*1

# Destroy effects / exile effects
# Note - are just treating destroy and exile as identical effects for now, for the purpose of getting a model working
#        ideally would split these up, and allow for some more nuance
# Note - need to go back and check interaction of nonland and permanent to make sure it is handle properly
df_extract_nonland = df.text.str.contains('([Dd]estroy|[Ee]xile)\s.*target.*?nonland(?=\.|\s)?')*1
df_extract_permanent = df.text.str.contains('([Dd]estroy|[Ee]xile)\s.*target.*(?<!nonland\s)permanent(?=\.|\s)?')*1

df_extract = df.text.str.contains('([Dd]estroy|[Ee]xile)\s.*target.*artifact?(\.|\s)')*1
df['effect_destroy_artifact'] = df_extract + df_extract_nonland + df_extract_permanent

df_extract = df.text.str.contains('([Dd]estroy|[Ee]xile)\s.*target.*creature?(\.|\s)')*1
df['effect_destroy_creature'] = df_extract + df_extract_nonland + df_extract_permanent

df_extract = df.text.str.contains('([Dd]estroy|[Ee]xile)\s.*target.*?(?<!is)(?<!non)land(?!walk)(?=\.|\s)?')*1
df['effect_destroy_land'] = df_extract + df_extract_permanent

df_extract = df.text.str.contains('([Dd]estroy|[Ee]xile)\s.*target.*?enchantment(?=\.|\s)?')*1
df['effect_destroy_enchantment'] = df_extract + df_extract_nonland + df_extract_permanent

df_extract = df.text.str.contains('([Dd]estroy|[Ee]xile)\s.*target.*?planeswalker(?=\.|\s)?')*1
df['effect_destroy_planeswalker'] = df_extract + df_extract_nonland + df_extract_permanent

# Destroying 'all' creatures
df_extract = df.text.str.contains('([Dd]estroy|[Ee]xile)\s.*all.*creatures(?=\.|\s)?')*1
df['effect_destroy_all_creatures'] = df_extract

# Deals damage effects
# Focus on damage to others, and excludes comabat damage to... triggers
df_extract = df.text.str.contains('(deals)+\s[\dX]*.*(?!combat\s)(damage)\sto(?!\syou)')
df['effect_deals_damage'] = df_extract*1

# Counter spell effects
df_extract = df.text.str.contains('[Cc]ounter.*spell')
df['effect_counter_target_spell'] = df_extract*1

# Enters the battlefield effect
df_extract = df.text.str.contains('[Ee]nter(s)?\sthe\sbattlefield')
df['effect_enter_the_battlefield'] = df_extract*1

df_extract = df.text.str.contains('[Ee]nter(s)?\sthe\sbattlefield.*[Ss]acrifice\sit')
df['effect_enter_the_battlefield_sacrific_it'] = df_extract*1

# Activate ability as an effect
df_extract = df.text.str.count('.*:.*')
df['effect_has_activated_ability'] = df_extract*1

############################################################
############################################################

# Set up base lines for efficiency metrics

# Power + Toughness 
# P+T Clean up power
df.power.fillna('{none}',inplace=True)
df['power_clean'] = 0
df['power_clean'] = np.where(df.power.str.contains('\D(?<![{noe}])'),1,0)
df['power_clean'] = [max(i/2,1) for i in df['convertedManaCost'].tolist()] * np.float64(df['power_clean'])
df['power_clean'] = np.where(df['power_clean']==0,df['power'],df['power_clean'])
df['power_clean'] = np.where(df['power_clean']=='{none}',0,df['power_clean'])

# P+T Clean up toughness

df.toughness.fillna('{none}',inplace=True)
df['toughness_clean'] = 0
df['toughness_clean'] = np.where(df.toughness.str.contains('\D(?<![{noe}])'),1,0)
df['toughness_clean'] = [max(i/2,1) for i in df['convertedManaCost'].tolist()] * np.float64(df['toughness_clean'])
df['toughness_clean'] = np.where(df['toughness_clean']==0,df['toughness'],df['toughness_clean'])
df['toughness_clean'] = np.where(df['toughness_clean']=='{none}',0,df['toughness_clean'])

# P+T Calculation
df['power_plus_toughness'] = np.float64(df['power_clean']) + np.float64(df['toughness_clean'])
df['power_plus_toughness']  = np.float64(df['power_plus_toughness'])

# Count keywords and effects
def sum_columns_starting_with(df, col_name_str):
    col_name_match_len = len(col_name_str)
    col_list = [i for i in df.columns.tolist() if str(i)[0:col_name_match_len] == col_name_str]
    sum_values = []
    temp_list = []
    for i in col_list:
        if sum_values==[]:
            sum_values = df[i].tolist()
        else:
            sum_values = [a+b for a,b in zip(sum_values, df[i])]
    return sum_values

df['keyword_count'] = sum_columns_starting_with(df,'keyword_')
df['effect_count'] = sum_columns_starting_with(df,'effect_')

# Calculate effieciency ratings
df['efficiency_power'] = np.where(df['convertedManaCost'].gt(0),np.float64(df['power_clean'])/df['convertedManaCost'],0)
df['efficiency_toughness'] = np.where(df['convertedManaCost'].gt(0),np.float64(df['toughness_clean'])/df['convertedManaCost'],0)
df['efficiency_p_plus_t'] = np.where(df['convertedManaCost'].gt(0),np.float64(df['power_plus_toughness'])/df['convertedManaCost'],0)
df['efficiency_keywords'] = np.where(df['convertedManaCost'].gt(0),np.float64(df['keyword_count'])/df['convertedManaCost'],0)
df['efficiency_effects'] = np.where(df['convertedManaCost'].gt(0),np.float64(df['effect_count'])/df['convertedManaCost'],0)

def max_columns_starting_with(df, col_name_str):
    col_name_match_len = len(col_name_str)
    col_list = [i for i in df.columns.tolist() if str(i)[0:col_name_match_len] == col_name_str]
    max_values = []
    temp_list = []
    for i in col_list:
        if max_values==[]:
            max_values = df[i].tolist()
        else:
            max_values = [max(a,b) for a,b in zip(max_values, df[i])]
    return max_values

df['efficiency_power'] = np.where((df.convertedManaCost==0) & ~(df['type'].str.contains("Land")),12,df['efficiency_power'])
df['efficiency_toughness'] = np.where((df.convertedManaCost==0) & ~(df['type'].str.contains("Land")),12,df['efficiency_toughness'])
df['efficiency_p_plus_t'] = np.where((df.convertedManaCost==0) & ~(df['type'].str.contains("Land")),12,df['efficiency_p_plus_t'])
df['efficiency_keywords'] = np.where((df.convertedManaCost==0) & ~(df['type'].str.contains("Land")),12,df['efficiency_keywords'])
df['efficiency_effects'] = np.where((df.convertedManaCost==0) & ~(df['type'].str.contains("Land")),12,df['efficiency_effects'])

df['efficiency_max'] = max_columns_starting_with(df,'efficiency_')

print('Card metrics data frame ready')


/usr/local/anaconda3/lib/python3.8/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


Card metrics data frame ready


# Draft Scores import summary

In [15]:
# Import draft scores

all_draftaholics_sets = [
    'AER','MM3','AKH','HOU','XLN',
    'IMA','UST','RIX','A25','DOM',
    'BBD','M19','GRN','UMA','RNA',
    'WAR','MH1','M20','ELD','MB1',
    'THB','IKO','CUB','M21','2XM',
    'AKR','ZNR','KLR','CMR']

def get_draft_scores(set_list):
    for i in set_list:
        req_path = "https://apps.draftaholicsanonymous.com/p1p1/" + i + "/results?ajax"
        results = requests.get(req_path)
        
        if results.status_code != 200:
            print(f'Was unable to retrieve data for {i}')
        else:
            results_json = json.loads(results.text)
            output_path = './draft-scores/scores_' + i + '.txt'
            with open(output_path,'w') as outfile:
                json.dump(results_json['data'],outfile)

                
def load_draft_scores_to_pandas(set_list):
    first_load = True
    for i in set_list:
        if first_load == True:
            first_load = False
            load_path = './draft-scores/scores_' + i + '.txt'
            df = pd.read_json(load_path)
            print(f"loaded {i} : {df['id'].count()}")
        else:
            load_path = './draft-scores/scores_' + i + '.txt' 
            df_next = pd.read_json(load_path)
            df = df.append(df_next,ignore_index=True)
            print(f"loaded {i} : {df_next['id'].count()} {df['id'].count()}")
    return df

# Note - only need to run get draft scores if we want to refresh data, otherwise
# assume to work from the underlying files (so can just load)
# get_draft_scores(all_draftaholics_sets)

df_scores = load_draft_scores_to_pandas(all_draftaholics_sets)


# Some different elo score conversions

df_scores['elo_log'] = np.log(df_scores['elo']) # Log conversion of elo
df_scores['elo_range_all'] = df_scores['elo'].max() - df_scores['elo'].min()

# Score relative to overall list of cards
# Note - Adding 1 to the top, and 2 to the denominator to present any score being precisely 0 or 1
df_scores['elo_relative_all'] = (df_scores['elo'] - df_scores['elo'].min() + 1) / (df_scores['elo_range_all'] + 2) 

# Score relative to all cards in a given set

elo_range_set = (df_scores.groupby('set_name')['elo'].max() - df_scores.groupby('set_name')['elo'].min()).reset_index()
elo_range_set.rename({'elo':'elo_range_set'},axis=1,inplace=True)
df_scores = df_scores.merge(elo_range_set,how='left',on='set_name')

# Set relative scores for all cards in a given set
elo_range_min = df_scores.groupby('set_name')['elo'].min().reset_index() 
elo_range_min.rename({'elo':'elo_set_min'},axis=1,inplace=True)
df_scores = df_scores.merge(elo_range_min,how='left',on='set_name')
df_scores['elo_relative_set'] = (df_scores['elo'] - df_scores['elo_set_min'] + 1) / (df_scores['elo_range_set'] + 2) 


# Drop unwanted columns for analysis

columns_to_drop = [
    # Items from website that aren't necessary
    'image_small',
    'image',
    'image_large',
    'back_image_small',
    'back_image',
    'exclude_from_p1p1',
    # Items to exclude as placeholders while building data
    'elo_range_all',
    'elo_range_set',
    'elo_set_min',
    ]

df_scores.drop(columns_to_drop,axis=1,inplace=True)

print('Card Scores dataframe ready.')


loaded AER : 184
loaded MM3 : 249 433
loaded AKH : 249 682
loaded HOU : 184 866
loaded XLN : 259 1125
loaded IMA : 249 1374
loaded UST : 214 1588
loaded RIX : 191 1779
loaded A25 : 249 2028
loaded DOM : 249 2277
loaded BBD : 249 2526
loaded M19 : 293 2819
loaded GRN : 254 3073
loaded UMA : 254 3327
loaded RNA : 254 3581
loaded WAR : 249 3830
loaded MH1 : 254 4084
loaded M20 : 260 4344
loaded ELD : 249 4593
loaded MB1 : 1694 6287
loaded THB : 249 6536
loaded IKO : 259 6795
loaded CUB : 555 7350
loaded M21 : 258 7608
loaded 2XM : 330 7938
loaded AKR : 303 8241
loaded ZNR : 265 8506
loaded KLR : 286 8792
loaded CMR : 361 9153
Card Scores dataframe ready.


In [16]:
df_scores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9153 entries, 0 to 9152
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                9153 non-null   int64  
 1   set_id            9153 non-null   int64  
 2   set_name          9153 non-null   object 
 3   name              9153 non-null   object 
 4   back_name         58 non-null     object 
 5   color             9153 non-null   object 
 6   elo               9153 non-null   int64  
 7   rank              9153 non-null   int64  
 8   rarity            9153 non-null   object 
 9   elo_log           9153 non-null   float64
 10  elo_relative_all  9153 non-null   float64
 11  elo_relative_set  9153 non-null   float64
dtypes: float64(3), int64(4), object(5)
memory usage: 929.6+ KB


# Add scores to data features

In [ ]:
# Create unique scores using averages for the first pass
df_scores.back_name.fillna('{none}',inplace=True)
df_scores_unique = df_scores.groupby(['name','back_name'])[['elo','elo_log','elo_relative_all','elo_relative_set']].mean().reset_index()
df_scores_unique.rename({'name':'front_name'},axis=1,inplace=True)

df_scores_unique['name'] = np.where(df_scores_unique.back_name=='{none}',df_scores_unique.front_name,df_scores_unique.front_name + ' // ' + df_scores_unique.back_name)
df = df.merge(df_scores_unique,on='name',how='left')

# Workings


In [4]:
page = requests.get("https://apps.draftaholicsanonymous.com/p1p1/M19")

In [6]:
page.content

b'<!DOCTYPE html>\n    <html lang="en">\n    <head>\n        <meta charset="utf-8">\n        <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n        <meta name="csrf-token" content="YEPGNv38cKmO1A99ryPulSctXXPi9jrJMxFiCyhr">\n        <title>P1P1 Core Set 2019 | Draftaholics Anonymous</title>\n        <meta name="description" content="Learn the cards. Start conversations. GET PICKING!">\n        <meta name="author" content="Draftaholics Anonymous">\n            <meta name="keywords"  content="magic,prerelease,p1p1,cards,mtg,gathering,draftaholics,draft,drafting,M19" />\n    <link rel="canonical" href="https://apps.draftaholicsanonymous.com/p1p1/M19" />\n    <meta property="og:title" content="P1P1 - Core Set 2019 | Draftaholics Anonymous" />\n    <meta property="og:type" content="article" />\n    <meta property="og:url" content="https://apps.draftaholicsanonymous.com/p1p1/M19" />\n    <meta property="og:image" content="https://apps.draftaholicsanon

In [8]:
soup = BeautifulSoup(page.content,'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <meta content="YEPGNv38cKmO1A99ryPulSctXXPi9jrJMxFiCyhr" name="csrf-token"/>
  <title>
   P1P1 Core Set 2019 | Draftaholics Anonymous
  </title>
  <meta content="Learn the cards. Start conversations. GET PICKING!" name="description"/>
  <meta content="Draftaholics Anonymous" name="author"/>
  <meta content="magic,prerelease,p1p1,cards,mtg,gathering,draftaholics,draft,drafting,M19" name="keywords">
   <link href="https://apps.draftaholicsanonymous.com/p1p1/M19" rel="canonical"/>
   <meta content="P1P1 - Core Set 2019 | Draftaholics Anonymous" property="og:title">
    <meta content="article" property="og:type">
     <meta content="https://apps.draftaholicsanonymous.com/p1p1/M19" property="og:url">
      <meta content="https://apps.draftaholicsanonymous.com/storage/M19.jpg" property="og:image">
       <meta content="Draftaholics Ano

In [9]:
list(soup.children)
# Looks like the key thing we need is the results call
# 'https://apps.draftaholicsanonymous.com/p1p1/M19/results?ajax'

['html',
 '\n',
 <html lang="en">
 <head>
 <meta charset="utf-8"/>
 <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
 <meta content="YEPGNv38cKmO1A99ryPulSctXXPi9jrJMxFiCyhr" name="csrf-token"/>
 <title>P1P1 Core Set 2019 | Draftaholics Anonymous</title>
 <meta content="Learn the cards. Start conversations. GET PICKING!" name="description"/>
 <meta content="Draftaholics Anonymous" name="author"/>
 <meta content="magic,prerelease,p1p1,cards,mtg,gathering,draftaholics,draft,drafting,M19" name="keywords">
 <link href="https://apps.draftaholicsanonymous.com/p1p1/M19" rel="canonical"/>
 <meta content="P1P1 - Core Set 2019 | Draftaholics Anonymous" property="og:title">
 <meta content="article" property="og:type">
 <meta content="https://apps.draftaholicsanonymous.com/p1p1/M19" property="og:url">
 <meta content="https://apps.draftaholicsanonymous.com/storage/M19.jpg" property="og:image">
 <meta content="Draftaholics Anonymous" property="og:site_name"/>
 

In [43]:
results = requests.get("https://apps.draftaholicsanonymous.com/p1p1/M19/results?ajax")

In [44]:
results.text

'{"data":[{"id":3367,"set_id":14,"set_name":"Core Set 2019","image_small":"\\/storage\\/cards\\/M19\\/small\\/34.jpg","image":"\\/storage\\/cards\\/M19\\/normal\\/34.jpg","image_large":"","back_image_small":"","back_image":"","name":"Resplendent Angel","back_name":null,"color":"white","elo":2229,"rank":1,"exclude_from_p1p1":null,"rarity":"mythic"},{"id":3171,"set_id":14,"set_name":"Core Set 2019","image_small":"\\/storage\\/cards\\/M19\\/small\\/3.jpg","image":"\\/storage\\/cards\\/M19\\/normal\\/3.jpg","image_large":"","back_image_small":"","back_image":"","name":"Ajani, Adversary of Tyrants","back_name":null,"color":"white","elo":2228,"rank":2,"exclude_from_p1p1":null,"rarity":"mythic"},{"id":3451,"set_id":14,"set_name":"Core Set 2019","image_small":"\\/storage\\/cards\\/M19\\/small\\/208.jpg","image":"\\/storage\\/cards\\/M19\\/normal\\/208.jpg","image_large":"","back_image_small":"","back_image":"","name":"Vivien Reid","back_name":null,"color":"green","elo":2143,"rank":3,"exclude_f

In [47]:
results_json = json.loads(results.text)

In [55]:
results_json['data'][0].keys()

dict_keys(['id', 'set_id', 'set_name', 'image_small', 'image', 'image_large', 'back_image_small', 'back_image', 'name', 'back_name', 'color', 'elo', 'rank', 'exclude_from_p1p1', 'rarity'])

In [56]:
results_json['data'][0]


{'id': 3367,
 'set_id': 14,
 'set_name': 'Core Set 2019',
 'image_small': '/storage/cards/M19/small/34.jpg',
 'image': '/storage/cards/M19/normal/34.jpg',
 'image_large': '',
 'back_image_small': '',
 'back_image': '',
 'name': 'Resplendent Angel',
 'back_name': None,
 'color': 'white',
 'elo': 2229,
 'rank': 1,
 'exclude_from_p1p1': None,
 'rarity': 'mythic'}

In [58]:
with open('draft_results.txt','w') as outfile:
    json.dump(results_json['data'],outfile)

In [60]:

df_p1p1 = pd.read_json('draft_results.txt')

In [63]:
df_p1p1.head()

,id,set_id,set_name,image_small,image,image_large,back_image_small,back_image,name,back_name,color,elo,rank,exclude_from_p1p1,rarity
0,3367,14,Core Set 2019,/storage/cards/M19/small/34.jpg,/storage/cards/M19/normal/34.jpg,,,,Resplendent Angel,None,white,2229,1,NaN,mythic
1,3171,14,Core Set 2019,/storage/cards/M19/small/3.jpg,/storage/cards/M19/normal/3.jpg,,,,"Ajani, Adversary of Tyrants",None,white,2228,2,NaN,mythic
2,3451,14,Core Set 2019,/storage/cards/M19/small/208.jpg,/storage/cards/M19/normal/208.jpg,,,,Vivien Reid,None,green,2143,3,NaN,mythic
3,3424,14,Core Set 2019,/storage/cards/M19/small/79.jpg,/storage/cards/M19/normal/79.jpg,,,,"Tezzeret, Artifice Master",None,blue,2135,4,NaN,mythic
4,3299,14,Core Set 2019,/storage/cards/M19/small/149.jpg,/storage/cards/M19/normal/149.jpg,,,,"Lathliss, Dragon Queen",None,red,2116,5,NaN,rare


In [62]:
df_p1p1.columns

Index(['id', 'set_id', 'set_name', 'image_small', 'image', 'image_large',
       'back_image_small', 'back_image', 'name', 'back_name', 'color', 'elo',
       'rank', 'exclude_from_p1p1', 'rarity'],
      dtype='object')

In [65]:
df_p1p1.drop(['image_small','image','image_large','back_image_small','back_image','exclude_from_p1p1'],axis=1, inplace=True)

In [66]:
df_p1p1.head()

,id,set_id,set_name,name,back_name,color,elo,rank,rarity
0,3367,14,Core Set 2019,Resplendent Angel,None,white,2229,1,mythic
1,3171,14,Core Set 2019,"Ajani, Adversary of Tyrants",None,white,2228,2,mythic
2,3451,14,Core Set 2019,Vivien Reid,None,green,2143,3,mythic
3,3424,14,Core Set 2019,"Tezzeret, Artifice Master",None,blue,2135,4,mythic
4,3299,14,Core Set 2019,"Lathliss, Dragon Queen",None,red,2116,5,rare


In [232]:
all_draftaholics_sets = [
    'AER','MM3','AKH','HOU','XLN',
    'IMA','UST','RIX','A25','DOM',
    'BBD','M19','GRN','UMA','RNA',
    'WAR','MH1','M20','ELD','MB1',
    'THB','IKO','CUB','M21','2XM',
    'AKR','ZNR','KLR','CMR']

In [73]:
results.status_code

200

In [74]:
!pwd

Start setting keys..
Keys set.
/Users/PRSmb/OneDrive/Documents/mtg-analytics/predict-draft-strength


In [87]:
def get_draft_scores(set_list):
    for i in set_list:
        req_path = "https://apps.draftaholicsanonymous.com/p1p1/" + i + "/results?ajax"
        results = requests.get(req_path)
        
        if results.status_code != 200:
            print(f'Was unable to retrieve data for {i}')
        else:
            results_json = json.loads(results.text)
            output_path = './draft-scores/scores_' + i + '.txt'
            with open(output_path,'w') as outfile:
                json.dump(results_json['data'],outfile)
            
    print("All sets added to files.")

In [233]:
get_draft_scores(all_draftaholics_sets)

All sets added to files.


In [234]:
def load_draft_scores_to_pandas(set_list):
    first_load = True
    for i in set_list:
        if first_load == True:
            first_load = False
            load_path = './draft-scores/scores_' + i + '.txt'
            df = pd.read_json(load_path)
            print(f"loaded {i} : {df['id'].count()}")
        else:
            load_path = './draft-scores/scores_' + i + '.txt' 
            df_next = pd.read_json(load_path)
            df = df.append(df_next,ignore_index=True)
            print(f"loaded {i} : {df_next['id'].count()} {df['id'].count()}")
    return df

In [129]:
df_test = load_draft_scores_to_pandas(['AER','AKH'])


loaded AER : 184
loaded AKH : 249 433'


In [235]:
df_scores = load_draft_scores_to_pandas(all_draftaholics_sets)

loaded AER : 184
loaded MM3 : 249 433
loaded AKH : 249 682
loaded HOU : 184 866
loaded XLN : 259 1125
loaded IMA : 249 1374
loaded UST : 214 1588
loaded RIX : 191 1779
loaded A25 : 249 2028
loaded DOM : 249 2277
loaded BBD : 249 2526
loaded M19 : 293 2819
loaded GRN : 254 3073
loaded UMA : 254 3327
loaded RNA : 254 3581
loaded WAR : 249 3830
loaded MH1 : 254 4084
loaded M20 : 260 4344
loaded ELD : 249 4593
loaded MB1 : 1694 6287
loaded THB : 249 6536
loaded IKO : 259 6795
loaded CUB : 555 7350
loaded M21 : 258 7608
loaded 2XM : 330 7938
loaded AKR : 303 8241
loaded ZNR : 265 8506
loaded KLR : 286 8792
loaded CMR : 361 9153


In [236]:
df_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9153 entries, 0 to 9152
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 9153 non-null   int64  
 1   set_id             9153 non-null   int64  
 2   set_name           9153 non-null   object 
 3   image_small        9153 non-null   object 
 4   image              9153 non-null   object 
 5   image_large        9153 non-null   object 
 6   back_image_small   9153 non-null   object 
 7   back_image         9153 non-null   object 
 8   name               9153 non-null   object 
 9   back_name          58 non-null     object 
 10  color              9153 non-null   object 
 11  elo                9153 non-null   int64  
 12  rank               9153 non-null   int64  
 13  exclude_from_p1p1  32 non-null     float64
 14  rarity             9153 non-null   object 
dtypes: float64(1), int64(4), object(10)
memory usage: 1.0+ MB


In [237]:
df_scores.rarity.value_counts()

common      3867
uncommon    2777
rare        1938
mythic       571
Name: rarity, dtype: int64

In [238]:
df_scores.groupby('set_name')['elo'].describe()

,count,mean,std,min,25%,50%,75%,max
set_name,,,,,,,,
Aether Revolt,184.0,1600.027174,248.875382,1082.0,1417.00,1563.5,1763.75,2210.0
Amonkhet,249.0,1600.004016,226.042647,1063.0,1432.00,1586.0,1755.00,2249.0
Amonkhet Remastered,303.0,1599.696370,240.804612,1091.0,1419.00,1596.0,1769.00,2196.0
Battlebond,249.0,1600.000000,174.772594,1161.0,1473.00,1584.0,1714.00,2075.0
Commander Legends,361.0,1600.000000,96.359570,1384.0,1527.00,1593.0,1672.00,1828.0
Core Set 2019,293.0,1600.010239,214.823239,1117.0,1465.00,1595.0,1725.00,2229.0
Core Set 2020,260.0,1599.473077,239.127797,1117.0,1425.75,1573.5,1764.00,2237.0
Core Set 2021,258.0,1600.003876,282.055649,997.0,1382.75,1569.5,1790.50,2330.0
Dominaria,249.0,1600.016064,256.435805,1059.0,1389.00,1585.0,1798.00,2164.0


In [239]:
name_counts = df_scores.groupby('name')['id'].count().reset_index()

In [240]:
name_counts.rename({'id':'count'},axis=1,inplace=True)

In [241]:
name_counts.sort_values(by='count',ascending=False,inplace=True)

In [242]:
name_counts.head(10)

,name,count
1908,Evolving Wilds,8
3935,Negate,7
1712,Duress,7
5202,Shock,7
4356,Plummet,7
497,Battle-Rattle Shaman,6
1230,Crash Through,6
1111,Colossal Dreadmaw,6
4199,Pacifism,6
31,Act of Treason,6


In [243]:
df_scores[(df_scores['name']=='Shock')][['set_name','elo']].sort_values(by='elo',ascending=False)

,set_name,elo
7389,Core Set 2021,1923
4139,Core Set 2020,1786
2596,Core Set 2019,1738
2343,Battlebond,1704
59,Aether Revolt,1699
6954,MTG Arena Draft Cube,1696
5002,Mystery Booster,1654


In [244]:
df_scores[(df_scores['name']=='Duress')][['set_name','elo']].sort_values(by='elo',ascending=False)

,set_name,elo
5883,Mystery Booster,1546
7197,MTG Arena Draft Cube,1482
2761,Core Set 2019,1418
4321,Core Set 2020,1313
1096,Ixalan,1308
1364,Iconic Masters,1258
7595,Core Set 2021,1183


# Initial Observations
* Cards appear multiple times across sets and have different values dependent on the set.
* This is to be expected given that different draft environments will drive different outcomes
* Need to do some checks on relative position / ranking for a given set
* **Question: Try to model specific sets or cards overall?**
    * A lot more value in cards overall given that this will allow more nuance in outcomes
    * Options on handling multiple scores - look at different metrics to convert the elo to a common score for a card. At least for the first rounds of modelling.

In [164]:
fig = px.box(df_scores,y='elo',x='rarity')
fig.show()

In [165]:
fig = px.box(df_scores,y='elo',x='set_name')
fig.show()

# elo conversions to build into data:
* Raw
* Relative rank within set (number 0 to 1)
* Relative rank across all cards (number 0 to 1)
* Log conversion

In [245]:
df_scores['elo_log'] = np.log(df_scores['elo']) # Log conversion of elo

In [246]:
fig = px.box(df_scores,y='elo_log',x='set_name')
fig.show()

In [247]:
df_scores['elo_range_all'] = df_scores['elo'].max() - df_scores['elo'].min()

In [248]:
# Note - Adding 1 to the top, and 2 to the denominator to present any score being precisely 0 or 1
df_scores['elo_relative_all'] = (df_scores['elo'] - df_scores['elo'].min() + 1) / (df_scores['elo_range_all'] + 2) 

In [249]:
df_scores['elo_relative_all'].describe()

count    9153.000000
mean        0.493385
std         0.133879
min         0.000646
25%         0.415752
50%         0.486120
75%         0.561653
max         0.999354
Name: elo_relative_all, dtype: float64

In [250]:
elo_range_set = (df_scores.groupby('set_name')['elo'].max() - df_scores.groupby('set_name')['elo'].min()).reset_index()

In [251]:
elo_range_set.rename({'elo':'elo_range_set'},axis=1,inplace=True)
elo_range_set.head()

,set_name,elo_range_set
0,Aether Revolt,1128
1,Amonkhet,1186
2,Amonkhet Remastered,1105
3,Battlebond,914
4,Commander Legends,444


In [252]:
df_scores = df_scores.merge(elo_range_set,how='left',on='set_name')

In [197]:
df_scores.drop(['elo_range_set'],inplace=True,axis=1)

In [253]:
df_scores.head(10)

,id,set_id,set_name,image_small,image,image_large,back_image_small,back_image,name,back_name,color,elo,rank,exclude_from_p1p1,rarity,elo_log,elo_range_all,elo_relative_all,elo_range_set
0,607,3,Aether Revolt,/storage/cards/AER/small/5e847c44-1849-4251-9075-88ed5c7792a6.jpg,/storage/cards/AER/normal/5e847c44-1849-4251-9075-88ed5c7792a6.jpg,,,,Heart of Kiran,NaN,colourless,2210,1,NaN,mythic,7.700748,1547,0.887024,1128
1,608,3,Aether Revolt,/storage/cards/AER/small/31a69ebe-4229-4067-8414-381b123fe63c.jpg,/storage/cards/AER/normal/31a69ebe-4229-4067-8414-381b123fe63c.jpg,,,,Herald of Anguish,NaN,black,2195,2,NaN,mythic,7.693937,1547,0.877340,1128
2,551,3,Aether Revolt,/storage/cards/AER/small/23b82ee9-2ac6-4b81-8c64-dd4f47c2d8cb.jpg,/storage/cards/AER/normal/23b82ee9-2ac6-4b81-8c64-dd4f47c2d8cb.jpg,,,,Ajani Unyielding,NaN,gold,2191,3,NaN,mythic,7.692113,1547,0.874758,1128
3,542,3,Aether Revolt,/storage/cards/AER/small/4160bb5f-4b49-4535-94f5-776d7abd1d1a.jpg,/storage/cards/AER/normal/4160bb5f-4b49-4535-94f5-776d7abd1d1a.jpg,,,,Aethersphere Harvester,NaN,colourless,2143,4,NaN,rare,7.669962,1547,0.843770,1128
4,718,3,Aether Revolt,/storage/cards/AER/small/329a8738-3e17-403a-857a-0ba529ce8cd1.jpg,/storage/cards/AER/normal/329a8738-3e17-403a-857a-0ba529ce8cd1.jpg,,,,Walking Ballista,NaN,colourless,2138,5,NaN,rare,7.667626,1547,0.840542,1128
5,708,3,Aether Revolt,/storage/cards/AER/small/58265203-bc16-41d2-875c-2ff3b4870824.jpg,/storage/cards/AER/normal/58265203-bc16-41d2-875c-2ff3b4870824.jpg,,,,Tezzeret the Schemer,NaN,gold,2106,6,NaN,mythic,7.652546,1547,0.819884,1128
6,726,3,Aether Revolt,/storage/cards/AER/small/f2f28735-122c-45ba-bde5-decfd9b11b32.jpg,/storage/cards/AER/normal/f2f28735-122c-45ba-bde5-decfd9b11b32.jpg,,,,Yahenni's Expertise,NaN,black,2083,7,NaN,rare,7.641564,1547,0.805036,1128
7,590,3,Aether Revolt,/storage/cards/AER/small/b5e81649-9954-424c-89d1-f87d73b66047.jpg,/storage/cards/AER/normal/b5e81649-9954-424c-89d1-f87d73b66047.jpg,,,,Fatal Push,NaN,black,2072,8,NaN,uncommon,7.636270,1547,0.797934,1128
8,680,3,Aether Revolt,/storage/cards/AER/small/7cff0dc6-5455-4dea-940b-dff7fe88dc5d.jpg,/storage/cards/AER/normal/7cff0dc6-5455-4dea-940b-dff7fe88dc5d.jpg,,,,"Rishkar, Peema Renegade",NaN,green,2070,9,NaN,rare,7.635304,1547,0.796643,1128
9,679,3,Aether Revolt,/storage/cards/AER/small/84b689cc-35ef-4a23-bb1e-4d81b9fb8455.jpg,/storage/cards/AER/normal/84b689cc-35ef-4a23-bb1e-4d81b9fb8455.jpg,,,,Ridgescale Tusker,NaN,green,2050,10,NaN,uncommon,7.625595,1547,0.783731,1128


In [254]:
elo_range_min = df_scores.groupby('set_name')['elo'].min().reset_index() 
elo_range_min.rename({'elo':'elo_set_min'},axis=1,inplace=True)
elo_range_min.head(5)

,set_name,elo_set_min
0,Aether Revolt,1082
1,Amonkhet,1063
2,Amonkhet Remastered,1091
3,Battlebond,1161
4,Commander Legends,1384


In [255]:
df_scores = df_scores.merge(elo_range_min,how='left',on='set_name')

In [256]:
df_scores['elo_relative_set'] = (df_scores['elo'] - df_scores['elo_set_min'] + 1) / (df_scores['elo_range_set'] + 2) 

In [222]:
# df_scores.info() #
#df_scores.drop('elo_relative_set',axis=1,inplace=True)
df_scores.drop('elo_y',axis=1,inplace=True)
df_scores.rename({'elo_x':'elo'},axis=1,inplace=True)

In [229]:

df_scores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7938 entries, 0 to 7937
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 7938 non-null   int64  
 1   set_id             7938 non-null   int64  
 2   set_name           7938 non-null   object 
 3   image_small        7938 non-null   object 
 4   image              7938 non-null   object 
 5   image_large        7938 non-null   object 
 6   back_image_small   7938 non-null   object 
 7   back_image         7938 non-null   object 
 8   name               7938 non-null   object 
 9   back_name          22 non-null     object 
 10  color              7938 non-null   object 
 11  elo                7938 non-null   int64  
 12  rank               7938 non-null   int64  
 13  exclude_from_p1p1  32 non-null     float64
 14  rarity             7938 non-null   object 
 15  elo_log            7938 non-null   float64
 16  elo_range_all      7938 

In [257]:
fig = px.box(df_scores,y='elo_relative_set',x='set_name')
fig.show()

In [258]:
df_scores[(df_scores['name']=='Duress')][['set_name','elo','elo_log','elo_relative_all','elo_relative_set']].sort_values(by='elo',ascending=False)

,set_name,elo,elo_log,elo_relative_all,elo_relative_set
5883,Mystery Booster,1546,7.343426,0.458360,0.420824
7197,MTG Arena Draft Cube,1482,7.301148,0.417043,0.389463
2761,Core Set 2019,1418,7.257003,0.375726,0.271095
4321,Core Set 2020,1313,7.180070,0.307941,0.175579
1096,Ixalan,1308,7.176255,0.304713,0.257403
1364,Iconic Masters,1258,7.137278,0.272434,0.094810
7595,Core Set 2021,1183,7.075809,0.224015,0.140075


In [8]:
df_scores.columns.tolist()

['id',
 'set_id',
 'set_name',
 'image_small',
 'image',
 'image_large',
 'back_image_small',
 'back_image',
 'name',
 'back_name',
 'color',
 'elo',
 'rank',
 'exclude_from_p1p1',
 'rarity',
 'elo_log',
 'elo_range_all',
 'elo_relative_all',
 'elo_range_set',
 'elo_set_min_x',
 'elo_relative_set',
 'elo_set_min_y']

For streamlit will want 3 data sets:
* Card Data Features
* Card Scoring outcomes (pared down)
* Scoring outcomes linked to data features (? force duplicates with sets - probably...)
* Scoring outcomes with an averaging function applied

In [48]:
# Create unique scores using averages for the first pass
df_scores.back_name.fillna('{none}',inplace=True)
df_scores_unique = df_scores.groupby(['name','back_name'])[['elo','elo_log','elo_relative_all','elo_relative_set']].mean().reset_index()
df_scores_unique.rename({'name':'front_name'},axis=1,inplace=True)


In [49]:
df_scores_unique.head()

,front_name,back_name,elo,elo_log,elo_relative_all,elo_relative_set
0,"""Rumors of My Death . . .""",{none},1443.0,7.274480,0.391866,0.159935
1,Abandoned Sarcophagus,{none},1512.5,7.321519,0.436733,0.280216
2,Abnormal Endurance,{none},1543.0,7.341484,0.456423,0.383303
3,Abominable Treefolk,{none},1847.0,7.521318,0.652679,0.692151
4,Abomination of Llanowar,{none},1661.0,7.415175,0.532602,0.623318


In [50]:
df_scores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9153 entries, 0 to 9152
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                9153 non-null   int64  
 1   set_id            9153 non-null   int64  
 2   set_name          9153 non-null   object 
 3   name              9153 non-null   object 
 4   back_name         9153 non-null   object 
 5   color             9153 non-null   object 
 6   elo               9153 non-null   int64  
 7   rank              9153 non-null   int64  
 8   rarity            9153 non-null   object 
 9   elo_log           9153 non-null   float64
 10  elo_relative_all  9153 non-null   float64
 11  elo_relative_set  9153 non-null   float64
dtypes: float64(3), int64(4), object(5)
memory usage: 929.6+ KB


In [31]:
df[df.name=='Akoum Warrior // Akoum Teeth'].name

55540    Akoum Warrior // Akoum Teeth
55541    Akoum Warrior // Akoum Teeth
Name: name, dtype: object

In [58]:
df_scores_unique['name'] = np.where(df_scores_unique.back_name=='{none}',df_scores_unique.front_name,df_scores_unique.front_name + ' // ' + df_scores_unique.back_name)

In [59]:
df_scores_unique.head()

,front_name,back_name,elo,elo_log,elo_relative_all,elo_relative_set,name
0,"""Rumors of My Death . . .""",{none},1443.0,7.274480,0.391866,0.159935,"""Rumors of My Death . . ."""
1,Abandoned Sarcophagus,{none},1512.5,7.321519,0.436733,0.280216,Abandoned Sarcophagus
2,Abnormal Endurance,{none},1543.0,7.341484,0.456423,0.383303,Abnormal Endurance
3,Abominable Treefolk,{none},1847.0,7.521318,0.652679,0.692151,Abominable Treefolk
4,Abomination of Llanowar,{none},1661.0,7.415175,0.532602,0.623318,Abomination of Llanowar


In [60]:
df_scores_unique[df_scores_unique.name=='Akoum Warrior // Akoum Teeth']

,front_name,back_name,elo,elo_log,elo_relative_all,elo_relative_set,name
122,Akoum Warrior,Akoum Teeth,1697.0,7.436617,0.555842,0.558266,Akoum Warrior // Akoum Teeth


In [66]:
df_scores_unique.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6777 entries, 0 to 6776
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   front_name        6777 non-null   object 
 1   back_name         6777 non-null   object 
 2   elo               6777 non-null   float64
 3   elo_log           6777 non-null   float64
 4   elo_relative_all  6777 non-null   float64
 5   elo_relative_set  6777 non-null   float64
 6   name              6777 non-null   object 
dtypes: float64(4), object(3)
memory usage: 370.7+ KB


In [62]:
df = df.merge(df_scores_unique,on='name',how='left')

In [63]:
df.head(10)

,index,id,colorIdentity,colorIndicator,colors,convertedManaCost,faceConvertedManaCost,faceName,flavorText,hand,hasAlternativeDeckLimit,isOnlineOnly,isOversized,isPromo,isReprint,isReserved,isStarter,isTextless,keywords,layout,leadershipSkills,life,loyalty,manaCost,multiverseId,name,number,otherFaceIds,power,printings,rarity,setCode,side,subtypes,supertypes,text,toughness,type,types,uuid,variations,watermark,name_row,double_layout,manaCost_NA,manaCost_Generic_count,manaCost_W_count,manaCost_U_count,manaCost_B_count,manaCost_R_count,manaCost_G_count,manaCost_C_count,manaCost_WP_count,manaCost_UP_count,manaCost_BP_count,manaCost_RP_count,manaCost_GP_count,manaCost_H_WU_count,manaCost_H_UB_count,manaCost_H_BR_count,manaCost_H_RG_count,manaCost_H_GW_count,manaCost_H_WB_count,manaCost_H_UR_count,manaCost_H_BG_count,manaCost_H_RW_count,manaCost_H_GU_count,manaCost_H_2W_count,manaCost_H_2U_count,manaCost_H_2B_count,manaCost_H_2R_count,manaCost_H_2G_count,manaCost_X_count,manaCost_Y_count,manaCost_Z_count,manaCost_Snow_count,manaCost_HW_count,keywords_NA,keywords_count,keyword_Adamant,keyword_Adapt,keyword_Addendum,keyword_Affinity,keyword_Afflict,keyword_Afterlife,keyword_Aftermath,keyword_Amass,keyword_Amplify,keyword_Annihilator,keyword_Ascend,keyword_Assemble,keyword_Assist,keyword_Augment,keyword_Aura_Swap,keyword_Awaken,keyword_Banding,keyword_Basic_landcycling,keyword_Battalion,keyword_Battle_Cry,keyword_Bestow,keyword_Bloodrush,keyword_Bloodthirst,keyword_Bolster,keyword_Bushido,keyword_Buyback,keyword_Cascade,keyword_Champion,keyword_Changeling,keyword_Channel,keyword_Chroma,keyword_Cipher,keyword_Clash,keyword_Cohort,keyword_Commander_ninjutsu,keyword_Companion,keyword_Conspire,keyword_Constellation,keyword_Converge,keyword_Convoke,keyword_Council's_dilemma,keyword_Crew,keyword_Cumulative_upkeep,keyword_Cycling,keyword_Dash,keyword_Deathtouch,keyword_Defender,keyword_Delirium,keyword_Delve,keyword_Desertwalk,keyword_Detain,keyword_Dethrone,keyword_Devoid,keyword_Devour,keyword_Domain,keyword_Double_strike,keyword_Dredge,keyword_Echo,keyword_Embalm,keyword_Emerge,keyword_Eminence,keyword_Enchant,keyword_Enrage,keyword_Entwine,keyword_Epic,keyword_Equip,keyword_Escalate,keyword_Escape,keyword_Eternalize,keyword_Evoke,keyword_Evolve,keyword_Exalted,keyword_Exert,keyword_Exploit,keyword_Explore,keyword_Extort,keyword_Fabricate,keyword_Fading,keyword_Fateful_hour,keyword_Fateseal,keyword_Fear,keyword_Ferocious,keyword_Fight,keyword_First_strike,keyword_Flanking,keyword_Flash,keyword_Flashback,keyword_Flying,keyword_Forecast,keyword_Forestcycling,keyword_Forestwalk,keyword_Formidable,keyword_Fortify,keyword_Frenzy,keyword_Fuse,keyword_Goad,keyword_Graft,keyword_Grandeur,keyword_Gravestorm,keyword_Haste,keyword_Haunt,keyword_Hellbent,keyword_Hero's_Reward,keyword_Heroic,keyword_Hexproof,keyword_Hexproof_from,keyword_Hidden_agenda,keyword_Hideaway,keyword_Horsemanship,keyword_Imprint,keyword_Improvise,keyword_Indestructible,keyword_Infect,keyword_Ingest,keyword_Inspired,keyword_Intimidate,keyword_Investigate,keyword_Islandcycling,keyword_Islandwalk,keyword_Join_forces,keyword_Jump-start,keyword_Kicker,keyword_Kinfall,keyword_Kinship,keyword_Landcycling,keyword_Landfall,keyword_Landship,keyword_Landwalk,keyword_Legacy,keyword_Legendary_landwalk,keyword_Level_Up,keyword_Lieutenant,keyword_Lifelink,keyword_Living_weapon,keyword_Madness,keyword_Manifest,keyword_Megamorph,keyword_Meld,keyword_Melee,keyword_Menace,keyword_Mentor,keyword_Metalcraft,keyword_Mill,keyword_Miracle,keyword_Modular,keyword_Monstrosity,keyword_Morbid,keyword_Morph,keyword_Mountaincycling,keyword_Mountainwalk,keyword_Multikicker,keyword_Mutate,keyword_Myriad,keyword_Ninjutsu,keyword_Nonbasic_landwalk,keyword_Outlast,keyword_Overload,keyword_Parley,keyword_Partner,keyword_Partner_with,keyword_Persist,keyword_Phasing,keyword_Plainscycling,keyword_Plainswalk,keyword_Populate,keyword_Proliferate,keyword_Protection,keyword_Provoke,keyword_Prowess,keyword_Prowl,keyword_R

In [65]:
df.describe()

,index,id,convertedManaCost,hand,hasAlternativeDeckLimit,isOnlineOnly,isOversized,isPromo,isReprint,isReserved,isStarter,isTextless,life,multiverseId,name_row,double_layout,manaCost_NA,manaCost_W_count,manaCost_U_count,manaCost_B_count,manaCost_R_count,manaCost_G_count,manaCost_C_count,manaCost_WP_count,manaCost_UP_count,manaCost_BP_count,manaCost_RP_count,manaCost_GP_count,manaCost_H_WU_count,manaCost_H_UB_count,manaCost_H_BR_count,manaCost_H_RG_count,manaCost_H_GW_count,manaCost_H_WB_count,manaCost_H_UR_count,manaCost_H_BG_count,manaCost_H_RW_count,manaCost_H_GU_count,manaCost_H_2W_count,manaCost_H_2U_count,manaCost_H_2B_count,manaCost_H_2R_count,manaCost_H_2G_count,manaCost_X_count,manaCost_Y_count,manaCost_Z_count,manaCost_Snow_count,manaCost_HW_count,keywords_NA,keywords_count,keyword_Adamant,keyword_Adapt,keyword_Addendum,keyword_Affinity,keyword_Afflict,keyword_Afterlife,keyword_Aftermath,keyword_Amass,keyword_Amplify,keyword_Annihilator,keyword_Ascend,keyword_Assemble,keyword_Assist,keyword_Augment,keyword_Aura_Swap,keyword_Awaken,keyword_Banding,keyword_Basic_landcycling,keyword_Battalion,keyword_Battle_Cry,keyword_Bestow,keyword_Bloodrush,keyword_Bloodthirst,keyword_Bolster,keyword_Bushido,keyword_Buyback,keyword_Cascade,keyword_Champion,keyword_Changeling,keyword_Channel,keyword_Chroma,keyword_Cipher,keyword_Clash,keyword_Cohort,keyword_Commander_ninjutsu,keyword_Companion,keyword_Conspire,keyword_Constellation,keyword_Converge,keyword_Convoke,keyword_Council's_dilemma,keyword_Crew,keyword_Cumulative_upkeep,keyword_Cycling,keyword_Dash,keyword_Deathtouch,keyword_Defender,keyword_Delirium,keyword_Delve,keyword_Desertwalk,keyword_Detain,keyword_Dethrone,keyword_Devoid,keyword_Devour,keyword_Domain,keyword_Double_strike,keyword_Dredge,keyword_Echo,keyword_Embalm,keyword_Emerge,keyword_Eminence,keyword_Enchant,keyword_Enrage,keyword_Entwine,keyword_Epic,keyword_Equip,keyword_Escalate,keyword_Escape,keyword_Eternalize,keyword_Evoke,keyword_Evolve,keyword_Exalted,keyword_Exert,keyword_Exploit,keyword_Explore,keyword_Extort,keyword_Fabricate,keyword_Fading,keyword_Fateful_hour,keyword_Fateseal,keyword_Fear,keyword_Ferocious,keyword_Fight,keyword_First_strike,keyword_Flanking,keyword_Flash,keyword_Flashback,keyword_Flying,keyword_Forecast,keyword_Forestcycling,keyword_Forestwalk,keyword_Formidable,keyword_Fortify,keyword_Frenzy,keyword_Fuse,keyword_Goad,keyword_Graft,keyword_Grandeur,keyword_Gravestorm,keyword_Haste,keyword_Haunt,keyword_Hellbent,keyword_Hero's_Reward,keyword_Heroic,keyword_Hexproof,keyword_Hexproof_from,keyword_Hidden_agenda,keyword_Hideaway,keyword_Horsemanship,keyword_Imprint,keyword_Improvise,keyword_Indestructible,keyword_Infect,keyword_Ingest,keyword_Inspired,keyword_Intimidate,keyword_Investigate,keyword_Islandcycling,keyword_Islandwalk,keyword_Join_forces,keyword_Jump-start,keyword_Kicker,keyword_Kinfall,keyword_Kinship,keyword_Landcycling,keyword_Landfall,keyword_Landship,keyword_Landwalk,keyword_Legacy,keyword_Legendary_landwalk,keyword_Level_Up,keyword_Lieutenant,keyword_Lifelink,keyword_Living_weapon,keyword_Madness,keyword_Manifest,keyword_Megamorph,keyword_Meld,keyword_Melee,keyword_Menace,keyword_Mentor,keyword_Metalcraft,keyword_Mill,keyword_Miracle,keyword_Modular,keyword_Monstrosity,keyword_Morbid,keyword_Morph,keyword_Mountaincycling,keyword_Mountainwalk,keyword_Multikicker,keyword_Mutate,keyword_Myriad,keyword_Ninjutsu,keyword_Nonbasic_landwalk,keyword_Outlast,keyword_Overload,keyword_Parley,keyword_Partner,keyword_Partner_with,keyword_Persist,keyword_Phasing,keyword_Plainscycling,keyword_Plainswalk,keyword_Populate,keyword_Proliferate,keyword_Protection,keyword_Provoke,keyword_Prowess,keyword_Prowl,keyword_Radiance,keyword_Raid,keyword_Rally,keyword_Rampage,keyword_Reach,keyword_Rebound,keyword_Recover,keyword_Reinforce,keyword_Renown,keyword_Replicate,keyword_Retrace,keyword_Revolt,keyword_Riot,keyword_Ripple,keyword_Scavenge,keyword_Scry,keyword_Shadow,keyword_Shroud,keyword_Skulk,key

***

This is probably the point to start setting up data exploration in streamlit / using streamlit and ploty charts 
Can start to look at interactions between data sets